In [119]:
import glob
import pandas as pd
import os
import json

import warnings
warnings.filterwarnings('ignore')

# Daily Data

In [65]:
inputdir = "../../data/website/databricks/hourly"
outputdir = "../../data/website/test/hourly"

problematicpaths = []

for tt in ["user", "spider"]:
    for at in ["desktop", "mobile-dev", "mobile-app"]:
        for domain in ["en.wikipedia", "de.wikipedia", "fr.wikipedia", "es.wikipedia", "ru.wikipedia", "zh.wikipedia"]:
            for year in [2019]:
                for month in [9]:
                    for day in range(1,32):

                        jsondic = {i:[] for i in range(24)}
                        filepathpattern = f"./{inputdir}/{tt}/{at}/{domain}/{year}-{month:02d}-{day:02d}-*.csv"
                        filepaths = glob.glob(filepathpattern)
                        
                        if not filepaths:
                            print(f"{filepathpattern} IS EMPTY")
                            problematicpaths.append(filepathpattern)
                            continue

                        elif len(filepaths) != 24:
                            print(f"{filepathpattern} DOES NOT CONTAIN 24 HOURS")
                            problematicpaths.append(filepathpattern)
                            continue
                        
                        for filepath in filepaths:
                            hour = int(filepath.split("-")[-1].split(".")[0])
                            nested_csvs = glob.glob(filepath + "/*.csv")
                            if len(nested_csvs) != 1:
                                print(f"{filepathpattern} CONTAINS NOT EXACTLY 1 NESTED CSV")
                                break
                            realfilepath = nested_csvs[0]
                            with open(realfilepath, "r") as f:
                                for line in f.read().split("\n"):
                                    x, y = line.split(",")
                                    jsondic[hour].append({"x":int(x), "y":int(y)})

                        outfilepath = f"./{outputdir}/{tt}/{at}/{domain}/{year}-{month:02d}-{day:02d}.json"
                        outfiledir = "/".join(outfilepath.split("/")[:-1])

                        if not os.path.exists(outfiledir):
                            os.makedirs(outfiledir)

                        with open(outfilepath, "w") as f:
                            json.dump(jsondic, f)

In [111]:
# !dbfs cp -r "dbfs:/mnt/group09/websitedata123/nice.parquet" "../../data/website/databricks/hourly2/sep2019.parquet"

In [177]:
df = pd.read_parquet("../../data/website/databricks/hourly2/sep2019.parquet").drop_duplicates()
df["date"] = pd.to_datetime(df["timestamp"], unit="s")
df["year"] = df.date.dt.year
df["month"] = df.date.dt.month
df["day"] = df.date.dt.day
df["hour"] = df.date.dt.hour
df

,timestamp,domain,trafficType,accessType,y,x,date,year,month,day,hour
0,1568192400,zh.wikipedia,user,desktop,2,143,2019-09-11 09:00:00,2019,9,11,9
1,1568199600,en.wikipedia,spider,desktop,20,6,2019-09-11 11:00:00,2019,9,11,11
2,1568199600,es.wikipedia,spider,desktop,845,1,2019-09-11 11:00:00,2019,9,11,11
3,1568196000,fr.wikipedia,spider,desktop,17,1,2019-09-11 10:00:00,2019,9,11,10
4,1568196000,fr.wikipedia,spider,desktop,19,1,2019-09-11 10:00:00,2019,9,11,10
...,...,...,...,...,...,...,...,...,...,...,...
851543,1567296000,de.wikipedia,user,desktop,27,1,2019-09-01 00:00:00,2019,9,1,0
851545,1567296000,en.wikipedia,spider,mobile-web,10,1,2019-09-01 00:00:00,2019,9,1,0
851563,1567296000,es.wikipedia,user,mobile-web,7,3,2019-09-01 00:00:00,2019,9,1,0
851572,1567296000,ru.wikipedia,user,mobile-web,3,10,2019-09-01 00:00:00,2019,9,1,0


In [181]:
def df_to_json(df):
    outputdir = "../../data/website/hourly"

    jsondic = {i:[] for i in range(24)}

    cumx = 0
    for _, (hour, x, y) in df[["hour", "x", "y"]].iterrows():
        cumx += x
        jsondic[hour].append({"x":int(cumx), "y":int(y)})

    tt = df["trafficType"].iloc[0]
    at = df["accessType"].iloc[0]
    domain = df["domain"].iloc[0]
    year = df["year"].iloc[0]
    month = df["month"].iloc[0]
    day = df["day"].iloc[0]
    print(day)
    print(df)

    outfilepath = f"./{outputdir}/{tt}/{at}/{domain}/{year}-{month:02d}-{day:02d}.json"
    outfiledir = "/".join(outfilepath.split("/")[:-1])

    if not os.path.exists(outfiledir):
        os.makedirs(outfiledir)

    with open(outfilepath, "w") as f:
        json.dump(jsondic, f)

In [183]:
# filtered_df = df[
#     (df["domain"] == "en.wikipedia") &
#     (df["accessType"] == "desktop") &
#     (df["trafficType"] == "user") &
#     (df["date"].dt.year == 2019) &
#     (df["date"].dt.month == 9)
# ]

df\
.sort_values(["trafficType", "accessType", "domain", "year", "month", "day", "hour", "y"], ascending=[False, False, False, True, True, True, True, False])\
.groupby(["trafficType", "accessType", "domain", "year", "month", "day"])\
.apply(df_to_json)

1
         timestamp        domain trafficType accessType      y      x  \
851327  1567296000  en.wikipedia        user    desktop  27205      1   
474317  1567296000  en.wikipedia        user    desktop    767      1   
644082  1567296000  en.wikipedia        user    desktop    590      1   
213049  1567296000  en.wikipedia        user    desktop    549      1   
723454  1567296000  en.wikipedia        user    desktop    545      1   
...            ...           ...         ...        ...    ...    ...   
775993  1567378800  en.wikipedia        user    desktop      5     52   
622506  1567378800  en.wikipedia        user    desktop      4    481   
170512  1567378800  en.wikipedia        user    desktop      3    380   
775590  1567378800  en.wikipedia        user    desktop      2   1628   
525063  1567378800  en.wikipedia        user    desktop      1  21912   

                      date  year  month  day  hour  
851327 2019-09-01 00:00:00  2019      9    1     0  
474317 2019-09-

""


In [123]:
outputdir = "../../data/website/hourly"

problematicpaths = []

prefilter_df = df
for tt in ["user", "spider"]:
    prefilter_df = prefilter_df[(df["trafficType"] == tt)]
    for at in ["desktop", "mobile-dev", "mobile-app"]:
        prefilter_df = prefilter_df[(df["accessType"] == at)]
        for domain in ["en.wikipedia", "de.wikipedia", "fr.wikipedia", "es.wikipedia", "ru.wikipedia", "zh.wikipedia"]:
            prefilter_df = prefilter_df[(df["domain"] == domain)]
            for year in [2019]:
                prefilter_df = prefilter_df[(df["date"].dt.year == year)]
                for month in [9]:
                    prefilter_df = prefilter_df[(df["date"].dt.month == month)]
                    for day in range(1,32):
                        prefilter_df = prefilter_df[(df["date"].dt.day == day)]

                        jsondic = {i:[] for i in range(24)}

                        print(1)
                        for hour in range(24):

                            # print(1)
                            filtered_df = prefilter_df[
                                (df["date"].dt.hour == hour)
                            ]

                            # filtered_df.sort_values("y", ascending=False, inplace=True)

                            # print(2)
                            cumx = 0
                            for _, (x, y) in filtered_df[["x", "y"]].iterrows():
                                cumx += x
                                jsondic[hour].append({"x":int(cumx), "y":int(y)})

                        print(3)
                        outfilepath = f"./{outputdir}/{tt}/{at}/{domain}/{year}-{month:02d}-{day:02d}.json"
                        outfiledir = "/".join(outfilepath.split("/")[:-1])

                        print(4)
                        if not os.path.exists(outfiledir):
                            os.makedirs(outfiledir)

                        print(5)
                        with open(outfilepath, "w") as f:
                            json.dump(jsondic, f)

                    8/0

1
3
4
5
1
3
4
5
1
3
4
5
1
3
4
5
1
3
4
5
1
3
4
5
1
3
4
5
1


KeyboardInterrupt: 

In [114]:
jsondic

{0: [{'x': 1, 'y': 27205},
  {'x': 1, 'y': 767},
  {'x': 1, 'y': 590},
  {'x': 1, 'y': 549},
  {'x': 1, 'y': 545},
  {'x': 1, 'y': 316},
  {'x': 1, 'y': 148},
  {'x': 1, 'y': 126},
  {'x': 1, 'y': 95},
  {'x': 1, 'y': 89},
  {'x': 1, 'y': 45},
  {'x': 1, 'y': 44},
  {'x': 1, 'y': 37},
  {'x': 1, 'y': 34},
  {'x': 1, 'y': 32},
  {'x': 1, 'y': 27},
  {'x': 2, 'y': 26},
  {'x': 1, 'y': 25},
  {'x': 2, 'y': 23},
  {'x': 1, 'y': 22},
  {'x': 2, 'y': 20},
  {'x': 5, 'y': 19},
  {'x': 1, 'y': 17},
  {'x': 2, 'y': 16},
  {'x': 1, 'y': 15},
  {'x': 2, 'y': 14},
  {'x': 4, 'y': 13},
  {'x': 4, 'y': 12},
  {'x': 6, 'y': 11},
  {'x': 3, 'y': 10},
  {'x': 7, 'y': 9},
  {'x': 58, 'y': 8},
  {'x': 9, 'y': 7},
  {'x': 25, 'y': 6},
  {'x': 41, 'y': 5},
  {'x': 321, 'y': 4},
  {'x': 306, 'y': 3},
  {'x': 1771, 'y': 2},
  {'x': 23850, 'y': 1}],
 1: [{'x': 1, 'y': 26805},
  {'x': 1, 'y': 26805},
  {'x': 1, 'y': 870},
  {'x': 1, 'y': 870},
  {'x': 1, 'y': 486},
  {'x': 1, 'y': 486},
  {'x': 1, 'y': 478},
 

In [106]:
filtered_df

,timestamp,domain,trafficType,accessType,y,x,date
32655,1567299600,en.wikipedia,user,desktop,19,2,2019-09-01 01:00:00
32705,1567299600,en.wikipedia,user,desktop,27,2,2019-09-01 01:00:00
66686,1567299600,en.wikipedia,user,desktop,21,2,2019-09-01 01:00:00
66740,1567299600,en.wikipedia,user,desktop,28,1,2019-09-01 01:00:00
66829,1567299600,en.wikipedia,user,desktop,486,1,2019-09-01 01:00:00
94007,1567299600,en.wikipedia,user,desktop,10,11,2019-09-01 01:00:00
94020,1567299600,en.wikipedia,user,desktop,6,25,2019-09-01 01:00:00
94106,1567299600,en.wikipedia,user,desktop,26,2,2019-09-01 01:00:00
94153,1567299600,en.wikipedia,user,desktop,478,1,2019-09-01 01:00:00
94167,1567299600,en.wikipedia,user,desktop,17,2,2019-09-01 01:00:00


# Monthly Data

In [71]:
# !dbfs cp -r "dbfs:/mnt/group09/websitedatamonthly/sep2019.parquet" "../../data/website/databricks/monthly/sep2019.parquet"

In [88]:
df = pd.read_parquet("../../data/website/databricks/monthly/sep2019.parquet")
df["date"] = pd.to_datetime(df["timestamp"], unit="s")
df

,timestamp,domain,trafficType,accessType,sumcount,date
0,1569204000,es.wikipedia,user,mobile-web,59591,2019-09-23 02:00:00
1,1567548000,zh.wikipedia,spider,mobile-web,1652,2019-09-03 22:00:00
2,1568260800,zh.wikipedia,user,mobile-app,21,2019-09-12 04:00:00
3,1568653200,en.wikipedia,spider,desktop,208181,2019-09-16 17:00:00
4,1569326400,es.wikipedia,spider,mobile-web,814,2019-09-24 12:00:00
...,...,...,...,...,...,...
21564,1569578400,en.wikipedia,spider,mobile-web,24723,2019-09-27 10:00:00
21565,1567720800,ru.wikipedia,spider,mobile-web,658,2019-09-05 22:00:00
21566,1568343600,en.wikipedia,user,mobile-web,50301,2019-09-13 03:00:00
21567,1569117600,es.wikipedia,user,desktop,6884,2019-09-22 02:00:00


In [90]:
outputdir = "../../data/website/monthly"

problematicpaths = []

for tt in ["user", "spider"]:
    for at in ["desktop", "mobile-dev", "mobile-app"]:
        for domain in ["en.wikipedia", "de.wikipedia", "fr.wikipedia", "es.wikipedia", "ru.wikipedia", "zh.wikipedia"]:
            for year in [2019]:
                for month in [9]:

                    jsondic = []

                    filtered_df = df[
                        (df["domain"] == domain) &
                        (df["accessType"] == at) &
                        (df["trafficType"] == tt) &
                        (df["date"].dt.year == year) &
                        (df["date"].dt.month == month)
                    ]

                    filtered_df.sort_values("timestamp", ascending=True, inplace=True)

                    for _, (date, y) in filtered_df[["date", "sumcount"]].iterrows():
                        x = f"{date.year:04d}-{date.month:02d}-{date.day:02d}-{date.hour:02d}"
                        jsondic.append({"x":x, "y":int(y)})

                    outfilepath = f"./{outputdir}/{tt}/{at}/{domain}/{year}-{month:02d}.json"
                    outfiledir = "/".join(outfilepath.split("/")[:-1])  # the directory where the file is in

                    if not os.path.exists(outfiledir):
                        os.makedirs(outfiledir)

                    with open(outfilepath, "w") as f:
                        json.dump(jsondic, f)